In [1]:
import metrics
import pandas as pd
import numpy as np
import tensorflow as tf

# Import test data

In [2]:
path_to_preprocessed_data = "C:/Users/vince/Documents/DataChallengeJFR/liver/pre_processed/"
dict_class = {"Malin":0, "Lesion":1, "Kyste":2,"Angiome":3, "CHC":4, "Foie Homogene":5, "HNF":6, "Metastase":7}

In [3]:
test_data = np.load(path_to_preprocessed_data + "test_data.npy")
test_labels = np.load(path_to_preprocessed_data + "test_labels.npy")

print("Test data shape: {}".format(test_data.shape))
print("Test label shape: {}".format(test_labels.shape))

Test data shape: (40, 691200)
Test label shape: (40, 8)


# Restore trained model

In [4]:
path_to_model = "C:/Users/vince/Documents/DataChallengeJFR/liver/model/"
model_name = "my_cnn_model"

In [5]:
sess=tf.Session()
#First let's load meta graph and restore weights
saver = tf.train.import_meta_graph(path_to_model + model_name + "/" + model_name + ".meta")
saver.restore(sess, tf.train.latest_checkpoint(path_to_model + model_name))

# Now, let's access and create placeholders variables and create feed-dict to feed new data
graph = tf.get_default_graph()
xin = graph.get_tensor_by_name("xin:0")
y_label = graph.get_tensor_by_name("y_label:0")
d = graph.get_tensor_by_name("d:0")

#Now, access the op that you want to run
y = graph.get_tensor_by_name("op_to_restore:0")

INFO:tensorflow:Restoring parameters from C:/Users/vince/Documents/DataChallengeJFR/liver/model/my_cnn_model/my_cnn_model


# Inference on test data & Export

In [6]:
path_to_inference = "C:/Users/vince/Documents/DataChallengeJFR/liver/inference/"

In [7]:
y_prob = sess.run(y, feed_dict={xin: test_data, y_label: test_labels, d: 1.0})
y_prob.shape

(40, 8)

In [8]:
df_pred = pd.DataFrame(y_prob, columns=list(dict_class.keys()))
df_pred.index.name = "id"
print(df_pred.head())

df_ref = pd.DataFrame(test_labels, columns=list(dict_class.keys()))
df_ref.index.name = "id"
print(df_ref.head())

df_pred.to_csv(path_to_inference + "prediction.csv")
df_ref.to_csv(path_to_inference + "reference.csv")

       Malin    Lésion     Kyste   Angiome       CHC  Foie Homogène       HNF  \
id                                                                              
0   0.022928  0.239743  0.011054  0.002790  0.002368       0.695763  0.002372   
1   0.015702  0.197550  0.006955  0.001564  0.001382       0.759754  0.001383   
2   0.014764  0.196419  0.006503  0.001369  0.001238       0.763694  0.001239   
3   0.021174  0.226904  0.009417  0.002218  0.001977       0.715088  0.001977   
4   0.016331  0.207452  0.007372  0.001586  0.001421       0.748062  0.001423   

    Métastase  
id             
0    0.022981  
1    0.015710  
2    0.014774  
3    0.021244  
4    0.016353  
    Malin  Lésion  Kyste  Angiome  CHC  Foie Homogène  HNF  Métastase
id                                                                   
0     0.0     1.0    1.0      0.0  0.0            0.0  0.0        0.0
1     0.0     0.0    0.0      0.0  0.0            1.0  0.0        0.0
2     0.0     0.0    0.0      0.0  0.0  

In [9]:
print(metrics.foie_metrics(path_to_inference + "reference.csv", path_to_inference + "prediction.csv"))

Only one class present in y_true. ROC AUC score is not defined in that case.
HNF
Only one class present in y_true. ROC AUC score is not defined in that case.
CHC
0.4763253968253968


# Score computation